# Neural Style Transfer

In this notebook, I'm mostly going to be playing around with neural style transfer using Keras. Hopefully, I can get something done.

File paths that will be used for the project. The image file is the 'content' file, the image I want to be transformed. The 'style' image is the image I will be using to steal the style from and applying it to the content image. The output image is obviously the output file.

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from keras import backend as K
from keras.preprocessing.image import load_iimg_to_arrayrray
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from scipy.optimize import fmin_l_bfgs_b
import time

Using TensorFlow backend.
/home/thibault/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
content = 'Images/dva.jpg'
style = 'Images/style1.jpg'
output = 'Images/output.jpg'

Library imports that will be used for image processing

In [3]:
from keras import backend as K
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

Some targets that the output image will have

In [4]:
height = 512
width = 512
size = (height, width)

Load up the content image

In [5]:
cimage = load_img(path = content, target_size = size)
cimArr = img_to_array(cimage)
cimArr = K.variable(preprocess_input(np.expand_dims(cimArr, axis = 0)), dtype = 'float32')

Load up the style image

In [6]:
simage = load_img(path = style, target_size = size)
simArr = img_to_array(simage)
simArr = K.variable(preprocess_input(np.expand_dims(simArr, axis = 0)), dtype = 'float32')

Finally I initialize the generated image

In [7]:
gimage0 = np.random.randint(256, size = (width, height, 3)).astype('float64')
gimage0 = preprocess_input(np.expand_dims(gimage0, axis = 0))
gPlaceholder = K.placeholder(shape = (1, width, height, 3))

Defining the content and style loss functions

In [8]:
def get_feature_reps(x, layer_names, model):
    featMatrices = []
    for ln in layer_names:
        selectedLayer = model.get_layer(ln)
        featRaw = selectedLayer.output
        featRawShape = K.shape(featRaw).eval(session = tf_session)
        N_1 = featRawShape[-1]
        M_1 = featRawShape[1] * featRawShape[2]
        featMatrix = K.reshape(featRaw, (M_1, N_1))
        featMatrix = K.transpose(featMatrix)
        featMatrices.append(featMatrix)
    
    return featMatrices

In [9]:
def get_content_loss(F, P):
    return 0.5*K.sum(K.square(F - P))

In [10]:
def get_gram_matrix(F):
    return K.dot(F, K.transpose(F))

In [11]:
def get_style_loss(ws, Gs, As):
    sLoss = K.variable(0.)
    
    for w, G, A in zip(ws, Gs, As):
        M_1 = K.int_shape(G)[1]
        N_1 = K.int_shape(G)[0]
        
        G_gram = get_gram_matrix(G)
        A_gram = get_gram_matrix(A)
        
        sLoss += w * 0.25 * K.sum(K.square(G_gram - A_gram)) / (N_1**2 * M_1**2)
        
    return sLoss

In [12]:
def get_total_loss(gImPlaceholder, alpha=1.0, beta=10000.0):
    F = get_feature_reps(gImPlaceholder, layer_names=[cLayerName], model=gModel)[0]
    Gs = get_feature_reps(gImPlaceholder, layer_names=sLayerNames, model=gModel)
    contentLoss = get_content_loss(F, P)
    styleLoss = get_style_loss(ws, Gs, As)
    totalLoss = alpha*contentLoss + beta*styleLoss
    return totalLoss

In [13]:
def calculate_loss(gImArr):
    """
    Calculate total loss using K.function
    """
    if gImArr.shape != (1, width, height, 3):
        gImArr = gImArr.reshape((1, width, height, 3))
        loss_fcn = K.function([gModel.input], [get_total_loss(gModel.input)])
        return loss_fcn([gImArr])[0].astype('float64')

def get_grad(gImArr):
    """
    Calculate the gradient of the loss function with respect to the generated image
    """
    if gImArr.shape != (1, width, height, 3):
        gImArr = gImArr.reshape((1, width, height, 3))
        grad_fcn = K.function([gModel.input], 
        K.gradients(get_total_loss(gModel.input), [gModel.input]))
        grad = grad_fcn([gImArr])[0].flatten().astype('float64')
        return grad

In [14]:
from keras.applications import VGG16
from scipy.optimize import fmin_l_bfgs_b

tf_session = K.get_session()
cModel = VGG16(include_top=False, weights='imagenet', input_tensor=cimArr)
sModel = VGG16(include_top=False, weights='imagenet', input_tensor=simArr)
gModel = VGG16(include_top=False, weights='imagenet', input_tensor=gPlaceholder)
cLayerName = 'block4_conv2'
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                ]

P = get_feature_reps(x=cimArr, layer_names=[cLayerName], model=cModel)[0]
As = get_feature_reps(x=simArr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))

iterations = 50
x_val = gimage0.flatten()
xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
                                maxiter=iterations, disp=True)

In [19]:
def postprocess_array(x):
    # Zero-center by mean pixel
    if x.shape != (width, height, 3):
        x = x.reshape((width, height, 3))
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

In [25]:
cImageOrig = Image.open(content)
cImageSizeOrig = cImageOrig.size

def save_original_size(x, target_size=cImageSizeOrig):
    xIm = Image.fromarray(x)
    xIm = xIm.resize(target_size)
    xIm.save(output)
    return xIm

In [27]:
xOut = postprocess_array(xopt)
xIm = save_original_size(xOut)
print('Image saved')

Image saved
